# Adversarial Robustness Framework: GTSRB Case Study
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Monthaonos/Advanced_machine_learning/blob/main/main.ipynb)

This interactive notebook demonstrates the adversarial vulnerability and defense mechanisms of deep learning models on the **GTSRB** (German Traffic Sign Recognition Benchmark) dataset.

### Experimental Pipeline:
1. **Phase 1: Training** - Standard and Adversarial (PGD) training for WideResNet/ResNet architectures.
2. **Phase 2: Evaluation** - Quantitative benchmarking of L-infinity robustness.
3. **Phase 3: Universal Patch Analysis** - Targeted L-0 attack using an optimized universal patch.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Clone the repository
!git clone https://github.com/Monthaonos/Advanced_machine_learning.git
%cd Advanced_machine_learning

# 2. Install dependencies
!pip install -q -r requirements.txt

# Set visual style
sns.set_theme(style="whitegrid")

In [ ]:
# Execute the orchestrator directly via CLI
# This uses your current main.py logic and priorities
!python main.py --train --eval --patch

In [ ]:
import toml

# Load config to dynamically find the correct files
config = toml.load("config.toml")
prefix = config["model"].get("prefix", "test0")
dataset = config["data"]["dataset"]
arch = config["model"]["architecture"]

# Path definitions
storage_path = config["project"]["storage_path"]
results_path = "results" # Using the standardized results folder

# --- 1. Plot Training Loss (Phase 1) ---
log_file = os.path.join(storage_path, f"{prefix}_{dataset}_{arch}_training_log.csv")

if os.path.exists(log_file):
    df_log = pd.read_csv(log_file)
    plt.figure(figsize=(10, 5))
    sns.lineplot(data=df_log, x='epoch', y='train_loss', marker='o', label='Training Loss')
    plt.title(f"Phase 1: Training Convergence ({prefix})")
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.show()

# --- 2. Plot Robustness Benchmark (Phase 2) ---
eval_file = os.path.join(results_path, f"{prefix}_{dataset}_{arch}_eval_report.csv")

if os.path.exists(eval_file):
    df_eval = pd.read_csv(eval_file)
    plt.figure(figsize=(10, 6))
    ax = sns.barplot(data=df_eval, x='Attack', y='Accuracy', hue='Model', palette='viridis')
    for container in ax.containers:
        ax.bar_label(container, fmt='%.1f%%', padding=3)
    plt.title("Phase 2: Accuracy under L-infinity Attacks")
    plt.ylabel("Accuracy (%)")
    plt.ylim(0, 110)
    plt.show()

# --- 3. Plot Patch Attack Success Rate (Phase 3) ---
patch_file = os.path.join(results_path, f"{prefix}_{dataset}_{arch}_patch_report.csv")

if os.path.exists(patch_file):
    df_patch = pd.read_csv(patch_file)
    plt.figure(figsize=(8, 6))
    ax_asr = sns.barplot(data=df_patch, x='Model', y='ASR', palette='magma')
    for container in ax_asr.containers:
        ax_asr.bar_label(container, fmt='%.2f%%', padding=3)
    plt.title("Phase 3: Universal Patch Attack Success Rate (ASR)")
    plt.ylabel("ASR (%)")
    plt.show()

In [ ]:
from IPython.display import Image, display

# Display the qualitative grid visualization generated during Phase 3
viz_path = os.path.join(results_path, f"{prefix}_{dataset}_{arch}_patch_viz.png")

if os.path.exists(viz_path):
    print("\n🎨 Qualitative Visualization (Patch Attack Samples):")
    display(Image(filename=viz_path))
else:
    print("[-] Visualization grid not found.")